In [ ]:
import folium
import json
import geopandas as gpd
import pandas as pd
import random
from shapely.geometry import Point, Polygon
from tqdm.notebook import trange, tqdm
import unidecode

Read file containing the cities polygons

In [ ]:
cities_gdf = gpd.read_file("cities/Case_studies_limits_final.shp")

In [ ]:
cities_gdf['Name']

Extract the selected entries corresponding to the selected cities

In [ ]:
def generate_random(number, polygon):
    points = []
    aux = polygon.bounds

    minx = aux[0]
    miny = aux[1]
    maxx = aux[2]
    maxy = aux[3]   
    
    while len(points) < number:
        pnt = Point(random.uniform(float(minx), float(maxx)), random.uniform(float(miny), float(maxy)))
        if polygon.contains(pnt):#.any():
            points.append(pnt)
    return points

In [ ]:
NUMBER_OF_POINTS_PER_ZONE = 100000

In [ ]:
for city_idx in trange(12, cities_gdf.shape[0], desc='Cities'): 
    city = cities_gdf.iloc[city_idx]
    
    columns = ['point_A', 'point_B']
    data = pd.DataFrame(columns=columns)
    
    for j in trange(NUMBER_OF_POINTS_PER_ZONE, desc='{} Points'.format(city.Name)): 
        points_or = generate_random(1, city.geometry)[0]
        points_de = generate_random(1, city.geometry)[0]

        data_row = {}
        data_row['point_A'] = [points_or.x, points_or.y]
        data_row['point_B'] = [points_de.x, points_de.y]

        data = data.append(data_row, ignore_index=True, sort=False)    
    
    OUTPUT_FILE = 'cities_points/points_{}.csv'.format(unidecode.unidecode(city.Name))
    data.to_csv(OUTPUT_FILE)

In [ ]:
m = folium.Map(location=[39, -8], zoom_start=4, tiles='CartoDB positron')


sim_geo = gpd.GeoSeries(city['geometry'])
geo_j = sim_geo.to_json()
geo_j = folium.GeoJson(data=geo_j,
                       style_function=lambda x: {'fillColor': 'orange'})
geo_j.add_to(m)

for _, row in data.iterrows():
    #print(row)
    folium.Marker([row['point_A'][1], row['point_A'][0]]).add_to(m)

m